In [10]:
import time
import nltk
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import os

In [11]:
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [12]:
os.chdir('C:/Users/user/Documents/Projet Statap/ neuroner/NeuroNER-master/data/conll2003/en')

In [13]:
train=pd.read_csv("train.txt", delim_whitespace=True, skipinitialspace=True, names= ['Word','POStag','CHUNKtag','NEtag'])
testa=pd.read_csv("valid.txt", delim_whitespace=True, skipinitialspace=True, names= ['Word','POStag','CHUNKtag','NEtag'])
testb=pd.read_csv("test.txt", delim_whitespace=True, skipinitialspace=True, names= ['Word','POStag','CHUNKtag','NEtag'])

In [14]:
print(train.shape,testa.shape,testb.shape)

(204567, 4) (51578, 4) (46666, 4)


In [15]:
train['CHUNKtag'].describe()
train = train.dropna()

In [16]:
train.shape

(202386, 4)

In [17]:
testa.head()

,Word,POStag,CHUNKtag,NEtag
0,-DOCSTART-,-X-,-X-,O
1,CRICKET,NNP,B-NP,O
2,-,:,O,O
3,LEICESTERSHIRE,NNP,B-NP,B-ORG
4,TAKE,NNP,I-NP,O


# Preprocess train

In [18]:
sparse = pd.get_dummies(train['NEtag'])



train['ORG'] = sparse['I-ORG'] + sparse['B-ORG']
train['LOC'] = sparse['I-LOC'] + sparse['B-LOC']
train['MISC'] = sparse['I-MISC'] + sparse['B-MISC']
train['PER'] = sparse['I-PER']

In [19]:
train

,Word,POStag,CHUNKtag,NEtag,ORG,LOC,MISC,PER
0,-DOCSTART-,-X-,-X-,O,0,0,0,0
1,EU,NNP,B-NP,B-ORG,1,0,0,0
2,rejects,VBZ,B-VP,O,0,0,0,0
3,German,JJ,B-NP,B-MISC,0,0,1,0
4,call,NN,I-NP,O,0,0,0,0
5,to,TO,B-VP,O,0,0,0,0
6,boycott,VB,I-VP,O,0,0,0,0
7,British,JJ,B-NP,B-MISC,0,0,1,0
8,lamb,NN,I-NP,O,0,0,0,0
9,.,.,O,O,0,0,0,0


In [20]:
gazloc = nltk.corpus.gazetteers.words(fileids=['countries.txt','uscities.txt','usstates.txt','usstateabbrev.txt','mexstates.txt','caprovinces.txt'])
gazper = nltk.corpus.names.words(fileids=['male.txt','female.txt'])
gazmisc = nltk.corpus.gazetteers.words(fileids=['nationalities.txt'])

In [21]:
len(gazmisc)

201

In [22]:
def gazetteer( text , ref ):
    res = []
    for word in text:
        if word in ref :
            res.append(1)
        else :
            res.append(0)
    return res


In [23]:
result = gazetteer(train['Word'] , gazloc)
x = train['LOC'] - result
print ('Real is ',sum(train['LOC']),' and we found ',sum(x),' locations')

Real is  8297  and we found  4445  locations


In [24]:
result = gazetteer(train['Word'] , gazper)
x = train['PER'] - result
print ('Real is ',sum(train['PER']),' and we found ',sum(x), ' persons')

Real is  4528  and we found  -1235  persons


In [25]:
result = gazetteer(train['Word'] , gazmisc)
x = train['MISC'] - result
print ('Real is ',sum(train['MISC']),' and we found ',sum(x), ' miscellaneous')

Real is  4593  and we found  3173  miscellaneous


Format BIO:

Before
Inside
Outside

In [26]:
def capitalize(text):
    cap = []
    for x in text:
        if x[0].isupper():
            cap.append(1)
        else : 
            cap.append(0)
    return cap

#capitalize(['.','EU','Germany','represents'])

In [27]:
def fullcap(text):
    cap = []
    for x in text:
        if x.isupper():
            cap.append(1)
        else : 
            cap.append(0)
    return cap

In [28]:
def length(text):
    length = []
    for x in text:
        length.append(len(x))
    return length

In [29]:
train['leng'] = length(train['Word']) 
train['Cap']=capitalize(train['Word'])
train['Cap2']=fullcap(train['Word'])
POStag = train['POStag'].tolist()
BPOStag = train['POStag'].tolist()
OPOStag = train['POStag'].tolist()

m = len(train['Word'])-1
for i in range(1,m):
    BPOStag[i] = POStag[i-1]
    OPOStag[i] = POStag[i+1]
    
train['BPOStag'] = BPOStag
train['OPOStag'] = OPOStag
train['GAZMISC'] = gazetteer(train['Word'] , gazmisc)
train['GAZPER'] = gazetteer(train['Word'] , gazper)
train['GAZLOC'] = gazetteer(train['Word'] , gazloc)

In [30]:
train.head()

,Word,POStag,CHUNKtag,NEtag,ORG,LOC,MISC,PER,leng,Cap,Cap2,BPOStag,OPOStag,GAZMISC,GAZPER,GAZLOC
0,-DOCSTART-,-X-,-X-,O,0,0,0,0,10,0,1,-X-,-X-,0,0,0
1,EU,NNP,B-NP,B-ORG,1,0,0,0,2,1,1,-X-,VBZ,0,0,0
2,rejects,VBZ,B-VP,O,0,0,0,0,7,0,0,NNP,JJ,0,0,0
3,German,JJ,B-NP,B-MISC,0,0,1,0,6,1,0,VBZ,NN,1,0,0
4,call,NN,I-NP,O,0,0,0,0,4,0,0,JJ,TO,0,0,0


# Regression logit

In [31]:
t=[]
for row in train.itertuples(index=True, name='Pandas'):
    if getattr(row,"CHUNKtag") =='O':
        t.append(0)
    
    else : 
        t.append(getattr(row,'Word').lower())
train['NoCaps']=t

In [32]:
prefixe=['anti','co','dis','il','im','in','inter','ir','mis','over','out','post','pre','pro','sub','super','trans','under']
suffixe=['dom','ship','hood','ian','er','er','ism','en','less','ish','ful','al','ly','en','ness','ship','ity','ize','ly']

def presufixe (x):
    if x == 0 :
        return 0
    if len(x)>5 :
        for j in range (0,6):
            if x[0:j] in prefixe :
                return 1
        for j in range(0,5):
            if x[len(x)-j:len(x)] in suffixe :
                return 1
    return 0
t=[]
for row in train.itertuples(index=True, name='Pandas'):
    t.append(presufixe(getattr(row,"NoCaps")))
train['presuf']=t
train

,Word,POStag,CHUNKtag,NEtag,ORG,LOC,MISC,PER,leng,Cap,Cap2,BPOStag,OPOStag,GAZMISC,GAZPER,GAZLOC,NoCaps,presuf
0,-DOCSTART-,-X-,-X-,O,0,0,0,0,10,0,1,-X-,-X-,0,0,0,-docstart-,0
1,EU,NNP,B-NP,B-ORG,1,0,0,0,2,1,1,-X-,VBZ,0,0,0,eu,0
2,rejects,VBZ,B-VP,O,0,0,0,0,7,0,0,NNP,JJ,0,0,0,rejects,0
3,German,JJ,B-NP,B-MISC,0,0,1,0,6,1,0,VBZ,NN,1,0,0,german,0
4,call,NN,I-NP,O,0,0,0,0,4,0,0,JJ,TO,0,0,0,call,0
5,to,TO,B-VP,O,0,0,0,0,2,0,0,NN,VB,0,0,0,to,0
6,boycott,VB,I-VP,O,0,0,0,0,7,0,0,TO,JJ,0,0,0,boycott,0
7,British,JJ,B-NP,B-MISC,0,0,1,0,7,1,0,VB,NN,1,0,0,british,1
8,lamb,NN,I-NP,O,0,0,0,0,4,0,0,JJ,.,0,0,0,lamb,0
9,.,.,O,O,0,0,0,0,1,0,0,NN,NNP,0,0,0,0,0


In [33]:
import collections

allname=[]
for row in train.itertuples(index=True, name='Pandas'):
    if getattr(row, 'NEtag')!="O":
        allname.append(getattr(row,'NoCaps'))
counter=collections.Counter(allname)
counter

Counter({'eu': 24,
         'german': 101,
         'british': 96,
         'peter': 31,
         'blackburn': 12,
         'brussels': 33,
         'european': 94,
         'commission': 49,
         'germany': 137,
         'union': 53,
         'werner': 2,
         'zwingmann': 2,
         'britain': 127,
         'nikolaus': 1,
         'van': 38,
         'der': 11,
         'pas': 1,
         'franz': 4,
         'fischler': 6,
         'eu-wide': 1,
         'france': 126,
         'bovine': 2,
         'spongiform': 2,
         'encephalopathy': 2,
         'bse': 11,
         'spanish': 15,
         'loyola': 1,
         'de': 57,
         'palacio': 1,
         'europe': 39,
         'welsh': 2,
         'national': 83,
         'farmers': 2,
         "'": 14,
         'nfu': 1,
         'john': 50,
         'lloyd': 21,
         'jones': 17,
         'bbc': 2,
         'radio': 7,
         'bonn': 28,
         'hendrix': 5,
         'london': 179,
         'u.s.': 377,
    

In [34]:
frequentname=[]
for i in allname:
    if counter[i]>=6 and i not in frequentname  :
        frequentname.append(i)

In [35]:
freq=[]
for row in train.itertuples(index=True, name='Pandas'):
    if getattr(row,"NoCaps") in frequentname :
        freq.append(1)
    else :
        freq.append(0)
train['Freq']=freq


In [36]:
allname=[]
for row in train.itertuples(index=True, name='Pandas'):
    if getattr(row, 'ORG')!=0:
        allname.append(getattr(row,'NoCaps'))
counter=collections.Counter(allname)

frequentname=[]
for i in allname:
    if counter[i]>=6 and i not in frequentname  :
        frequentname.append(i)
        
freq=[]
for row in train.itertuples(index=True, name='Pandas'):
    if getattr(row,"NoCaps") in frequentname :
        freq.append(1)
    else :
        freq.append(0)
train['Freqorg']=freq

In [37]:
allname=[]
for row in train.itertuples(index=True, name='Pandas'):
    if getattr(row, 'LOC')!=0:
        allname.append(getattr(row,'NoCaps'))
counter=collections.Counter(allname)

frequentname=[]
for i in allname:
    if counter[i]>=6 and i not in frequentname  :
        frequentname.append(i)
        
freq=[]
for row in train.itertuples(index=True, name='Pandas'):
    if getattr(row,"NoCaps") in frequentname :
        freq.append(1)
    else :
        freq.append(0)
train['Freqloc']=freq

In [38]:
allname=[]
for row in train.itertuples(index=True, name='Pandas'):
    if getattr(row, 'PER')!=0:
        allname.append(getattr(row,'NoCaps'))
counter=collections.Counter(allname)

frequentname=[]
for i in allname:
    if counter[i]>=6 and i not in frequentname  :
        frequentname.append(i)
        
freq=[]
for row in train.itertuples(index=True, name='Pandas'):
    if getattr(row,"NoCaps") in frequentname :
        freq.append(1)
    else :
        freq.append(0)
train['Freqper']=freq

In [39]:
allname=[]
for row in train.itertuples(index=True, name='Pandas'):
    if getattr(row, 'MISC')!=0:
        allname.append(getattr(row,'NoCaps'))
counter=collections.Counter(allname)

frequentname=[]
for i in allname:
    if counter[i]>=6 and i not in frequentname  :
        frequentname.append(i)
        
freq=[]
for row in train.itertuples(index=True, name='Pandas'):
    if getattr(row,"NoCaps") in frequentname :
        freq.append(1)
    else :
        freq.append(0)
train['Freqmisc']=freq

In [40]:
Numb=['0','1','2','3','4','5','6','7','8','9']
nube=[]
def nub(x):
    if x==0:
        return 0
    for i in Numb :
        if i in x :
            return 1
    return 0
for row in train.itertuples(index=True, name='Pandas'):
    nube.append(nub(getattr(row, "NoCaps")))
train['Number']=nube


In [41]:
alpha=['ORG','MISC','LOC','PER']
a='per'+alpha[2]
a

'perLOC'

In [42]:
for at in alpha:
    uni=[]
    for i,row in enumerate(train.itertuples(index=True, name='Pandas'),1):
        if getattr(row,at)==1 :
            uni.append(train.iloc[i-2,16])
    counter2=collections.Counter(uni)

    frequentuni=[]
    for i in uni:
        if counter2[i]>=6 and i not in frequentuni  :
            frequentuni.append(i)
    L=[0]
    for i in range (1,len(train)):
        if train.iloc[i-1,16] in frequentuni :
            L.append(1)
        else :
            L.append(0)
    train['UNI'+at]=L

        

In [43]:
for at in alpha:
    postuni=[]
    for i,row in enumerate(train.itertuples(index=True, name='Pandas'),1):
        if getattr(row,at)==1 :
            postuni.append(train.iloc[i,16])
    counter2=collections.Counter(postuni)

    frequentuni=[]
    for i in postuni:
        if counter2[i]>=6 and i not in frequentuni  :
            frequentuni.append(i)
    L=[]
    for i in range (0,len(train)-1):
        if train.iloc[i+1,16] in frequentuni :
            L.append(1)
        else :
            L.append(0)
    L.append(0)
    train['POSTUNI'+at]=L



In [44]:
train

,Word,POStag,CHUNKtag,NEtag,ORG,LOC,MISC,PER,leng,Cap,...,Freqmisc,Number,UNIORG,UNIMISC,UNILOC,UNIPER,POSTUNIORG,POSTUNIMISC,POSTUNILOC,POSTUNIPER
0,-DOCSTART-,-X-,-X-,O,0,0,0,0,10,0,...,0,0,0,0,0,0,0,0,0,0
1,EU,NNP,B-NP,B-ORG,1,0,0,0,2,1,...,0,0,1,1,1,0,0,0,0,0
2,rejects,VBZ,B-VP,O,0,0,0,0,7,0,...,0,0,0,0,0,0,0,0,0,0
3,German,JJ,B-NP,B-MISC,0,0,1,0,6,1,...,1,0,0,0,0,0,0,0,0,0
4,call,NN,I-NP,O,0,0,0,0,4,0,...,0,0,0,1,0,0,1,1,1,1
5,to,TO,B-VP,O,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
6,boycott,VB,I-VP,O,0,0,0,0,7,0,...,0,0,1,1,1,0,0,0,0,0
7,British,JJ,B-NP,B-MISC,0,0,1,0,7,1,...,1,0,0,0,0,0,0,0,0,0
8,lamb,NN,I-NP,O,0,0,0,0,4,0,...,0,0,1,1,0,0,1,1,1,1
9,.,.,O,O,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0


In [45]:
train.columns

Index(['Word', 'POStag', 'CHUNKtag', 'NEtag', 'ORG', 'LOC', 'MISC', 'PER',
       'leng', 'Cap', 'Cap2', 'BPOStag', 'OPOStag', 'GAZMISC', 'GAZPER',
       'GAZLOC', 'NoCaps', 'presuf', 'Freq', 'Freqorg', 'Freqloc', 'Freqper',
       'Freqmisc', 'Number', 'UNIORG', 'UNIMISC', 'UNILOC', 'UNIPER',
       'POSTUNIORG', 'POSTUNIMISC', 'POSTUNILOC', 'POSTUNIPER'],
      dtype='object')

On va effectuer les 4 regression logit

In [46]:
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

Sur les organisations

In [47]:
Liste=['Cap','presuf','Freq','Freqorg', 'Freqloc', 'Freqper',
       'Freqmisc', 'Number', 'UNIORG', 'UNIMISC', 'UNILOC', 'UNIPER',
       'POSTUNIORG', 'POSTUNIMISC', 'POSTUNILOC', 'POSTUNIPER','GAZMISC', 'GAZPER',
       'GAZLOC']
y=['ORG']

In [48]:
logreg = LogisticRegression()
rfe = RFE(logreg, 2)
rfe = rfe.fit(train[Liste], train[y].values.ravel() )
print(rfe.support_)
print(rfe.ranking_)

[ True False False False False False False False  True False False False
 False False False False False False False]
[ 1 18 12  3  9 15  4  7  1  8  2 14  5 10  6 17 13 11 16]


In [49]:
import statsmodels.api as sm

C:\Users\user\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [50]:
YORG=train['ORG']
XORG=train[Liste]
logit_modelORG=sm.Logit(YORG,XORG)
resultORG=logit_modelORG.fit()
print(resultORG.summary())
XORG_train, XORG_test, YORG_train, YORG_test = train_test_split(XORG, YORG, test_size=0.3, random_state=0)
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logregORG = LogisticRegression()
logregORG.fit(XORG_train, YORG_train)
YORG_pred = logregORG.predict(XORG_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logregORG.score(XORG_test, YORG_test)))
metrics.confusion_matrix(YORG_test, YORG_pred, labels=None, sample_weight=None)
print(metrics.classification_report(YORG_test, YORG_pred))

C:\Users\user\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\user\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1264: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


Optimization terminated successfully.
         Current function value: inf
         Iterations 11


C:\Users\user\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:2397: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.llf/self.llnull
C:\Users\user\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:2401: RuntimeWarning: invalid value encountered in double_scalars
  return -2*(self.llnull - self.llf)
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


                           Logit Regression Results                           
Dep. Variable:                    ORG   No. Observations:               202386
Model:                          Logit   Df Residuals:                   202367
Method:                           MLE   Df Model:                           18
Date:                Wed, 04 Apr 2018   Pseudo R-squ.:                     nan
Time:                        21:34:28   Log-Likelihood:                   -inf
converged:                       True   LL-Null:                          -inf
                                        LLR p-value:                       nan
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Cap             1.5087      0.023     65.250      0.000       1.463       1.554
presuf         -1.5961      0.037    -43.000      0.000      -1.669      -1.523
Freq           -2.3507      0.055    -42.594    

In [51]:
def Lesorgas(texte):
    df=preprocess(texte)
    dftest=df[['Cap', 'presuf', 'Freq', 'Freqorg', 'Freqloc', 'Freqper', 'Freqmisc',
       'Number', 'UNIORG', 'UNIMISC', 'UNILOC', 'UNIPER', 'POSTUNIORG',
       'POSTUNIMISC', 'POSTUNILOC', 'POSTUNIPER', 'GAZMISC', 'GAZPER',
       'GAZLOC']].copy()
    Yba = logregORG.predict(dftest)
    L=[]
    for i in range (0, len(Yba)):
        if Yba[i]!=0:
            L.append(df.iloc[[i],[0]])
    return L
    

Personnes


In [52]:
Liste=['Cap','presuf','Freq','Freqorg', 'Freqloc', 'Freqper',
       'Freqmisc', 'Number', 'UNIORG', 'UNIMISC', 'UNILOC', 'UNIPER',
       'POSTUNIORG', 'POSTUNIMISC', 'POSTUNILOC', 'POSTUNIPER','GAZMISC', 'GAZPER',
       'GAZLOC']
yPER=['PER']
logreg = LogisticRegression()
rfe = RFE(logreg, 2)
rfe = rfe.fit(train[Liste], train[yPER].values.ravel() )
print(rfe.support_)
print(rfe.ranking_)
YPER=train['PER']
XPER=train[Liste]
logit_modelPER=sm.Logit(YPER,XPER)
resultPER=logit_modelPER.fit()
print(resultPER.summary())
XPER_train, XPER_test, YPER_train, YPER_test = train_test_split(XPER, YPER, test_size=0.3, random_state=0)
logregPER = LogisticRegression()
logregPER.fit(XPER_train, YPER_train)
YPER_pred = logregPER.predict(XPER_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logregPER.score(XPER_test, YPER_test)))
metrics.confusion_matrix(YPER_test, YPER_pred, labels=None, sample_weight=None)

[ True False False False False False False False False  True False False
 False False False False False False False]
[ 1 17 15  8 14 10  9  4  3  1  7  2 12 13 16  6 11 18  5]


C:\Users\user\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\user\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1264: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


         Current function value: inf
         Iterations: 35


C:\Users\user\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\user\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:2397: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.llf/self.llnull
C:\Users\user\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:2401: RuntimeWarning: invalid value encountered in double_scalars
  return -2*(self.llnull - self.llf)
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818:

                           Logit Regression Results                           
Dep. Variable:                    PER   No. Observations:               202386
Model:                          Logit   Df Residuals:                   202367
Method:                           MLE   Df Model:                           18
Date:                Wed, 04 Apr 2018   Pseudo R-squ.:                     nan
Time:                        21:34:46   Log-Likelihood:                   -inf
converged:                      False   LL-Null:                          -inf
                                        LLR p-value:                       nan
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Cap             0.3271      0.033      9.950      0.000       0.263       0.392
presuf         -2.4422      0.065    -37.314      0.000      -2.570      -2.314
Freq           -1.1797      0.069    -17.209    

array([[59362,    42],
       [  622,   690]], dtype=int64)

In [53]:
print(metrics.classification_report(YPER_test, YPER_pred))

             precision    recall  f1-score   support

          0       0.99      1.00      0.99     59404
          1       0.94      0.53      0.68      1312

avg / total       0.99      0.99      0.99     60716



In [54]:
def Lepersonnes(texte):
    df=preprocess(texte)
    dftest=df[['Cap', 'presuf', 'Freq', 'Freqorg', 'Freqloc', 'Freqper', 'Freqmisc',
       'Number', 'UNIORG', 'UNIMISC', 'UNILOC', 'UNIPER', 'POSTUNIORG',
       'POSTUNIMISC', 'POSTUNILOC', 'POSTUNIPER', 'GAZMISC', 'GAZPER',
       'GAZLOC']].copy()
    Yba = logregPER.predict(dftest)
    L=[]
    for i in range (0, len(Yba)):
        if Yba[i]!=0:
            L.append(df.iloc[[i],[0]])
    return L
    

Lieux

In [55]:
Liste=['Cap','presuf','Freq','Freqorg', 'Freqloc', 'Freqper',
       'Freqmisc', 'Number', 'UNIORG', 'UNIMISC', 'UNILOC', 'UNIPER',
       'POSTUNIORG', 'POSTUNIMISC', 'POSTUNILOC', 'POSTUNIPER','GAZMISC', 'GAZPER',
       'GAZLOC']
yLOC=['LOC']
logreg = LogisticRegression()
rfe = RFE(logreg, 2)
rfe = rfe.fit(train[Liste], train[yLOC].values.ravel() )
print(rfe.support_)
print(rfe.ranking_)
YLOC=train['LOC']
XLOC=train[Liste]
logit_modelLOC=sm.Logit(YLOC,XLOC)
resultLOC=logit_modelLOC.fit()
print(resultLOC.summary())
XLOC_train, XLOC_test, YLOC_train, YLOC_test = train_test_split(XLOC, YLOC, test_size=0.3, random_state=0)
logregLOC = LogisticRegression()
logregLOC.fit(XLOC_train, YLOC_train)
YLOC_pred = logregLOC.predict(XLOC_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logregLOC.score(XLOC_test, YLOC_test)))
metrics.confusion_matrix(YLOC_test, YLOC_pred, labels=None, sample_weight=None)

[ True False False False  True False False False False False False False
 False False False False False False False]
[ 1 15 14  9  1  4  8 11  6 16  5 13  7 17  3 18 10 12  2]


C:\Users\user\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\user\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1264: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


Optimization terminated successfully.
         Current function value: inf
         Iterations 13


C:\Users\user\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:2397: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.llf/self.llnull
C:\Users\user\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:2401: RuntimeWarning: invalid value encountered in double_scalars
  return -2*(self.llnull - self.llf)
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


                           Logit Regression Results                           
Dep. Variable:                    LOC   No. Observations:               202386
Model:                          Logit   Df Residuals:                   202367
Method:                           MLE   Df Model:                           18
Date:                Wed, 04 Apr 2018   Pseudo R-squ.:                     nan
Time:                        21:35:01   Log-Likelihood:                   -inf
converged:                       True   LL-Null:                          -inf
                                        LLR p-value:                       nan
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Cap             0.1228      0.027      4.488      0.000       0.069       0.176
presuf         -2.7279      0.066    -41.302      0.000      -2.857      -2.598
Freq           -1.6832      0.069    -24.554    

array([[57981,   216],
       [  610,  1909]], dtype=int64)

In [56]:
print(metrics.classification_report(YLOC_test, YLOC_pred))

             precision    recall  f1-score   support

          0       0.99      1.00      0.99     58197
          1       0.90      0.76      0.82      2519

avg / total       0.99      0.99      0.99     60716



In [57]:
def Leslieux(texte):
    df=preprocess(texte)
    dftest=df[['Cap', 'presuf', 'Freq', 'Freqorg', 'Freqloc', 'Freqper', 'Freqmisc',
       'Number', 'UNIORG', 'UNIMISC', 'UNILOC', 'UNIPER', 'POSTUNIORG',
       'POSTUNIMISC', 'POSTUNILOC', 'POSTUNIPER', 'GAZMISC', 'GAZPER',
       'GAZLOC']].copy()
    Yba = logregLOC.predict(dftest)
    L=[]
    for i in range (0, len(Yba)):
        if Yba[i]!=0:
            L.append(df.iloc[[i],[0]])
    return L
    

Miscealeanous

In [58]:
Liste=['Cap','presuf','Freq','Freqorg', 'Freqloc', 'Freqper',
       'Freqmisc', 'Number', 'UNIORG', 'UNIMISC', 'UNILOC', 'UNIPER',
       'POSTUNIORG', 'POSTUNIMISC', 'POSTUNILOC', 'POSTUNIPER','GAZMISC', 'GAZPER',
       'GAZLOC']
yMISC=['MISC']
logreg = LogisticRegression()
rfe = RFE(logreg, 2)
rfe = rfe.fit(train[Liste], train[yMISC].values.ravel() )
print(rfe.support_)
print(rfe.ranking_)
YMISC=train['MISC']
XMISC=train[Liste]
logit_model=sm.Logit(YMISC,XMISC)
result=logit_model.fit()
print(result.summary())
XMISC_train, XMISC_test, YMISC_train, YMISC_test = train_test_split(XMISC, YMISC, test_size=0.3, random_state=0)
logregMISC = LogisticRegression()
logregMISC.fit(XMISC_train, YMISC_train)
YMISC_pred = logregMISC.predict(XMISC_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logregMISC.score(XMISC_test, YMISC_test)))
metrics.confusion_matrix(YMISC_test, YMISC_pred, labels=None, sample_weight=None)

[ True False False False False False  True False False False False False
 False False False False False False False]
[ 1 13 17 12  2 16  1  4  9  8 18  7  6  5 11 15  3 10 14]


C:\Users\user\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\user\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1264: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


Optimization terminated successfully.
         Current function value: inf
         Iterations 10


C:\Users\user\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:2397: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.llf/self.llnull
C:\Users\user\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:2401: RuntimeWarning: invalid value encountered in double_scalars
  return -2*(self.llnull - self.llf)
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


                           Logit Regression Results                           
Dep. Variable:                   MISC   No. Observations:               202386
Model:                          Logit   Df Residuals:                   202367
Method:                           MLE   Df Model:                           18
Date:                Wed, 04 Apr 2018   Pseudo R-squ.:                     nan
Time:                        21:35:14   Log-Likelihood:                   -inf
converged:                       True   LL-Null:                          -inf
                                        LLR p-value:                       nan
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Cap             0.5554      0.029     19.395      0.000       0.499       0.612
presuf         -1.7924      0.048    -37.717      0.000      -1.886      -1.699
Freq           -3.7715      0.118    -31.832    

array([[59121,   181],
       [  446,   968]], dtype=int64)

In [59]:
print(metrics.classification_report(YMISC_test, YMISC_pred))

             precision    recall  f1-score   support

          0       0.99      1.00      0.99     59302
          1       0.84      0.68      0.76      1414

avg / total       0.99      0.99      0.99     60716



In [60]:
def Lesmisc(texte):
    df=preprocess(texte)
    dftest=df[['Cap', 'presuf', 'Freq', 'Freqorg', 'Freqloc', 'Freqper', 'Freqmisc',
       'Number', 'UNIORG', 'UNIMISC', 'UNILOC', 'UNIPER', 'POSTUNIORG',
       'POSTUNIMISC', 'POSTUNILOC', 'POSTUNIPER', 'GAZMISC', 'GAZPER',
       'GAZLOC']].copy()
    Yba = logregMISC.predict(dftest)
    L=[]
    for i in range (0, len(Yba)):
        if Yba[i]!=0:
            L.append(df.iloc[[i],[0]])
    return L
    

# On crée une fonction finale

In [61]:
def Final(texte):
    ORG=Lesorgas(texte)
    LOC=Leslieux(texte)
    PER=Lepersonnes(texte)
    MISC=Lesmisc(texte)
    ORGLOC=list(set(ORG).intersection(LOC))
    ORGPER=list(set(ORG).intersection(PER))
    ORGMISC=list(set(ORG).intersection(MISC))
    PERLOC=list(set(PER).intersection(LOC))
    MISCLOC=list(set(MISC).intersection(LOC))
    PERMISC=list(set(PER).intersection(MISC))
    print("Les organisations sont " + ORG)
    print("Les lieux sont" + LOC)
    print("Les personnes sont " +  PER)
    print("Les MISC sont " + MISC)
    print("Les éléments appartenant à plusieurs liste sont" + ORGLOC+ ORGPER +ORGMISC+ PERLOC+ MISCLOC+ PERMISC)

# PROBIT MULTINOMIALE

In [62]:
train

,Word,POStag,CHUNKtag,NEtag,ORG,LOC,MISC,PER,leng,Cap,...,Freqmisc,Number,UNIORG,UNIMISC,UNILOC,UNIPER,POSTUNIORG,POSTUNIMISC,POSTUNILOC,POSTUNIPER
0,-DOCSTART-,-X-,-X-,O,0,0,0,0,10,0,...,0,0,0,0,0,0,0,0,0,0
1,EU,NNP,B-NP,B-ORG,1,0,0,0,2,1,...,0,0,1,1,1,0,0,0,0,0
2,rejects,VBZ,B-VP,O,0,0,0,0,7,0,...,0,0,0,0,0,0,0,0,0,0
3,German,JJ,B-NP,B-MISC,0,0,1,0,6,1,...,1,0,0,0,0,0,0,0,0,0
4,call,NN,I-NP,O,0,0,0,0,4,0,...,0,0,0,1,0,0,1,1,1,1
5,to,TO,B-VP,O,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
6,boycott,VB,I-VP,O,0,0,0,0,7,0,...,0,0,1,1,1,0,0,0,0,0
7,British,JJ,B-NP,B-MISC,0,0,1,0,7,1,...,1,0,0,0,0,0,0,0,0,0
8,lamb,NN,I-NP,O,0,0,0,0,4,0,...,0,0,1,1,0,0,1,1,1,1
9,.,.,O,O,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0


In [63]:
multi=[]
for i in range (0,len(train)):
    if train.iloc[i,4]==1 :
        multi.append(1)
    elif train.iloc[i,5]==1 : 
        multi.append(2)
    elif train.iloc[i,6]==1 : 
        multi.append(3)
    elif train.iloc[i,7]==1 : 
        multi.append(4)
    else :
        multi.append(0)
train['multi']=multi

In [64]:
from sklearn import svm

X=train[['Cap', 'presuf', 'Freq', 'Freqorg', 'Freqloc', 'Freqper', 'Freqmisc',
       'Number', 'UNIORG', 'UNIMISC', 'UNILOC', 'UNIPER', 'POSTUNIORG',
       'POSTUNIMISC', 'POSTUNILOC', 'POSTUNIPER', 'GAZMISC', 'GAZPER',
       'GAZLOC']].copy()
y=multi
C = 1.0

In [65]:
# SVC with linear kernel
svc = svm.SVC(kernel='linear', C=C).fit(X, y)


In [69]:
svc.predict(X[0:10])

array([0, 1, 0, 3, 0, 0, 0, 3, 0, 0])

In [70]:
train

,Word,POStag,CHUNKtag,NEtag,ORG,LOC,MISC,PER,leng,Cap,...,Number,UNIORG,UNIMISC,UNILOC,UNIPER,POSTUNIORG,POSTUNIMISC,POSTUNILOC,POSTUNIPER,multi
0,-DOCSTART-,-X-,-X-,O,0,0,0,0,10,0,...,0,0,0,0,0,0,0,0,0,0
1,EU,NNP,B-NP,B-ORG,1,0,0,0,2,1,...,0,1,1,1,0,0,0,0,0,1
2,rejects,VBZ,B-VP,O,0,0,0,0,7,0,...,0,0,0,0,0,0,0,0,0,0
3,German,JJ,B-NP,B-MISC,0,0,1,0,6,1,...,0,0,0,0,0,0,0,0,0,3
4,call,NN,I-NP,O,0,0,0,0,4,0,...,0,0,1,0,0,1,1,1,1,0
5,to,TO,B-VP,O,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
6,boycott,VB,I-VP,O,0,0,0,0,7,0,...,0,1,1,1,0,0,0,0,0,0
7,British,JJ,B-NP,B-MISC,0,0,1,0,7,1,...,0,0,0,0,0,0,0,0,0,3
8,lamb,NN,I-NP,O,0,0,0,0,4,0,...,0,1,1,0,0,1,1,1,1,0
9,.,.,O,O,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C).fit(X, y)

In [74]:
rbf_svc.score(X,y)

0.95635567677606159

In [76]:
testa

,Word,POStag,CHUNKtag,NEtag
0,-DOCSTART-,-X-,-X-,O
1,CRICKET,NNP,B-NP,O
2,-,:,O,O
3,LEICESTERSHIRE,NNP,B-NP,B-ORG
4,TAKE,NNP,I-NP,O
5,OVER,IN,B-PP,O
6,AT,NNP,B-NP,O
7,TOP,NNP,I-NP,O
8,AFTER,NNP,I-NP,O
9,INNINGS,NNP,I-NP,O


# Preprocess test

In [261]:
word = []
for x in testa['Word']:
    word.append(str(x))

testa['Word']=word

In [257]:
testa=preprocess(testa)
testa

In [259]:
print(testa)

None


In [262]:
testa['leng'] = length(testa['Word']) 
testa['Cap']=capitalize(testa['Word'])
testa['Cap2']=fullcap(testa['Word'])
POStag = testa['POStag'].tolist()
BPOStag = testa['POStag'].tolist()
OPOStag = testa['POStag'].tolist()

m = len(testa['Word'])-1
for i in range(1,m):
    BPOStag[i] = POStag[i-1]
    OPOStag[i] = POStag[i+1]
    
testa['BPOStag'] = BPOStag
testa['OPOStag'] = OPOStag
testa['GAZMISC'] = gazetteer(testa['Word'] , gazmisc)
testa['GAZPER'] = gazetteer(testa['Word'] , gazper)
testa['GAZLOC'] = gazetteer(testa['Word'] , gazloc)

preprocess(testa)

In [263]:
POStag = testa['POStag'].tolist()

for index, item in enumerate(POStag):
    if item == 'O' :
        POStag[index] = '-X-'
        
testa['POStag'] = POStag

In [264]:
POStag = testa['POStag'].tolist()
BPOStag = testa['POStag'].tolist()
OPOStag = testa['POStag'].tolist()

m = len(testa['Word'])-1
for i in range(1,m):
    BPOStag[i] = POStag[i-1]
    OPOStag[i] = POStag[i+1]
    
testa['BPOStag'] = BPOStag
testa['OPOStag'] = OPOStag

In [265]:
sparse = pd.get_dummies(testa['NEtag'])
sparse.head()

,B-LOC,B-MISC,B-ORG,B-PER,I-LOC,I-MISC,I-ORG,I-PER,O
0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1


In [266]:
sparse = pd.get_dummies(testa['NEtag'])

testa['ORG'] = sparse['I-ORG'] + sparse['B-ORG']
testa['LOC'] = sparse['I-LOC'] +sparse['B-LOC']
testa['MISC'] = sparse['I-MISC'] + sparse['B-MISC']
testa['PER'] = sparse['I-PER']+sparse['B-PER']

In [287]:
testa.head()

,Word,POStag,CHUNKtag,NEtag,leng,Cap,Cap2,BPOStag,OPOStag,GAZMISC,GAZPER,GAZLOC,ORG,LOC,MISC,PER
0,-DOCSTART-,-X-,-X-,O,10,0,1,-X-,-X-,0,0,0,0,0,0,0
1,CRICKET,NNP,B-NP,O,7,1,1,-X-,:,0,0,0,0,0,0,0
2,-,:,O,O,1,0,0,NNP,NNP,0,0,0,0,0,0,0
3,LEICESTERSHIRE,NNP,B-NP,B-ORG,14,1,1,:,NNP,0,0,0,1,0,0,0
4,TAKE,NNP,I-NP,O,4,1,1,NNP,IN,0,0,0,0,0,0,0


In [318]:
datatest = testa.drop(['Word','NEtag','ORG','LOC','MISC','PER'], axis=1)
datatest

,POStag,CHUNKtag,leng,Cap,Cap2,BPOStag,OPOStag,GAZMISC,GAZPER,GAZLOC
0,-X-,-X-,10,0,1,-X-,-X-,0,0,0
1,NNP,B-NP,7,1,1,-X-,:,0,0,0
2,:,O,1,0,0,NNP,NNP,0,0,0
3,NNP,B-NP,14,1,1,:,NNP,0,0,0
4,NNP,I-NP,4,1,1,NNP,IN,0,0,0
5,IN,B-PP,4,1,1,NNP,NNP,0,0,0
6,NNP,B-NP,2,1,1,IN,NNP,0,0,0
7,NNP,I-NP,3,1,1,NNP,NNP,0,0,0
8,NNP,I-NP,5,1,1,NNP,NNP,0,0,0
9,NNP,I-NP,7,1,1,NNP,NN,0,0,0


In [319]:
sparse = pd.get_dummies(datatest.iloc[:,[0,1,5,6]])
datatest.drop(['POStag','OPOStag','BPOStag','CHUNKtag'] , axis=1 , inplace=True)
datatest = pd.concat([datatest , sparse] , axis=1)

In [320]:
targettest = testa.iloc[:,12:16]

# Prédiction

In [321]:
data = train.drop(['Word','NEtag','ORG','LOC','MISC','PER'], axis=1)
data

,POStag,CHUNKtag,leng,Cap,Cap2,BPOStag,OPOStag,GAZMISC,GAZPER,GAZLOC
0,-X-,-X-,10,0,1,-X-,-X-,0,0,0
1,NNP,B-NP,2,1,1,-X-,VBZ,0,0,0
2,VBZ,B-VP,7,0,0,NNP,JJ,0,0,0
3,JJ,B-NP,6,1,0,VBZ,NN,1,0,0
4,NN,I-NP,4,0,0,JJ,TO,0,0,0
5,TO,B-VP,2,0,0,NN,VB,0,0,0
6,VB,I-VP,7,0,0,TO,JJ,0,0,0
7,JJ,B-NP,7,1,0,VB,NN,1,0,0
8,NN,I-NP,4,0,0,JJ,.,0,0,0
9,.,O,1,0,0,NN,NNP,0,0,0


In [322]:
sparse = pd.get_dummies(data.iloc[:,[0,1,5,6]])
data.drop(['POStag','OPOStag','BPOStag','CHUNKtag'] , axis=1 , inplace=True)
data = pd.concat([data , sparse] , axis=1)

In [323]:
target = train.iloc[:,4:8]

In [324]:
targettest

,ORG,LOC,MISC,PER
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,1,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0


In [325]:
start = time.time()
model = RandomForestClassifier()
model.fit(data , target)
end = time.time()
print(model.score(data , target))
predict = model.predict(datatest)
print(metrics.accuracy_score(targettest, predict))
print ('time :', end - start)

0.951355330902
0.89394703168
time : 6.979624271392822


In [326]:
print(metrics.classification_report(targettest , predict))

             precision    recall  f1-score   support

          0       0.66      0.37      0.48      2092
          1       0.86      0.57      0.69      2094
          2       0.80      0.49      0.61      1268
          3       0.78      0.22      0.34      3145

avg / total       0.77      0.38      0.50      8599



In [328]:
metrics.confusion_matrix(targettest, predict, labels=None, sample_weight=None)

ValueError: multilabel-indicator is not supported

In [327]:
print(sum(predict))
print(sum(targettest['ORG']), sum(targettest['LOC']), sum(targettest['MISC']), sum(['PER']))

[ 1174.  1396.   779.   889.]


TypeError: unsupported operand type(s) for +: 'int' and 'str'

Gazetteer ConLL
Passer le truc en script
Fabriquer des gazetteers dans d'autres langues

In [297]:
def capitalize(text):
    cap = []
    for x in text:
        if x[0].isupper():
            cap.append(1)
        else : 
            cap.append(0)
    return cap

def fullcap(text):
    cap = []
    for x in text:
        if x.isupper():
            cap.append(1)
        else : 
            cap.append(0)
    return cap

def length(text):
    length = []
    for x in text:
        length.append(len(x))
    return length

def gazetteer( text , ref ):
    res = []
    for word in text:
        if word in ref :
            res.append(1)
        else :
            res.append(0)
    return res

def preprocess(data):
    data['Cap']=capitalize(data['Word'])
    data['Cap2']=fullcap(data['Word'])
    data['leng'] = length(data['Word'])
    data['GAZMISC'] = gazetteer(data['Word'] , gazmisc)
    data['GAZPER'] = gazetteer(data['Word'] , gazper)
    data['GAZLOC'] = gazetteer(data['Word'] , gazloc) 
    return data

In [ ]:
start = time.time()
mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='saga').fit(words, NE)
print(mul_lr.score(words , sparse))
end = time.time()
print ('time :', end - start)